In [7]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
from timeit import default_timer as timer
from _collections import OrderedDict
import shutil
from pprint import pprint
start = timer()
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)
from utilities.file_location import FileLocationManager
from utilities.sqlcontroller import SqlController
from utilities.utilities_cvat_neuroglancer import get_structure_number, NumpyToNeuroglancer, get_segment_properties
from utilities.utilities_affine import align_point_sets, load_atlas_data, estimate_structure_centers


In [74]:
animal = 'DK52'
fileLocationManager = FileLocationManager(animal)
OUTPUT_DIR = os.path.join(fileLocationManager.neuroglancer_data, 'atlas')
if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)
os.makedirs(OUTPUT_DIR, exist_ok=True)

atlas_data = load_atlas_data()
source_centers = estimate_structure_centers(atlas_data, animal)
# DK52 structure centers
output_centers = {
    '12N': [46488, 18778, 242],
    '5N_L': [38990, 20019, 172],
    '5N_R': [39184, 19027, 315],
    '7N_L': [42425, 23190, 166],
    '7N_R': [42286, 22901, 291]
}
structures = sorted(output_centers.keys())
source_point_set = np.array([source_centers[s][0] for s in structures]).T
# Unify to 1 mum scale in all axes
source_scale = np.diagflat([10, 10, 20])
#source_point_set = source_scale @ source_point_set
output_point_set = np.array([output_centers[s] for s in structures]).T

# Unify to 1 mum scale in all axes
output_scale = np.diagflat([0.325, 0.325, 20])
#output_point_set = output_scale @ output_point_set

r_auto, t_auto = align_point_sets(source_point_set, output_point_set)
t_auto_out = (np.linalg.inv(output_scale) @ t_auto).T


resolution: 0.325
width: 65000
height: 36000
box center: [1056.25  585.    242.5 ]


In [56]:
print(source_point_set)
print(output_point_set)

[[1260.05026858 1010.78141394 1010.78141394 1088.27028024 1088.27028024]
 [ 655.12536632  587.58009809  587.58009809  705.01534472  705.01534472]
 [ 220.25         91.25519758  365.74480242  103.60451566  335.89548434]]
[[46488 38990 39184 42425 42286]
 [18778 20019 19027 23190 22901]
 [  242   172   315   166   291]]


In [67]:
sql_controller = SqlController(animal)
thumbnail_dir = os.path.join(fileLocationManager.prep, 'CH1', 'thumbnail_aligned')

print(f'resolution: {sql_controller.scan_run.resolution}')
print(f'width: {sql_controller.scan_run.width}')
print(f'height: {sql_controller.scan_run.height}')
box_w = sql_controller.scan_run.width * sql_controller.scan_run.resolution / 10  # 10 mum scale
box_h = sql_controller.scan_run.height * sql_controller.scan_run.resolution / 10  # 10 mum scale
box_z = len(os.listdir(thumbnail_dir))  # 20 mum scale
box_center = np.array([box_w, box_h, box_z]) / 2

atlasV7_volume = np.zeros((int(box_h), int(box_w), int(box_z)), dtype=np.uint8)
print('Shape of atlas volume', atlasV7_volume.shape)


resolution: 0.325
width: 65000
height: 36000
Shape of atlas volume (1170, 2112, 485)


In [62]:
t_auto.T

array([[23395.2914148 , 18823.65039869, -4757.22956042]])

In [75]:
for structure, (source_point, volume) in sorted(source_centers.items()):
    print(str(structure).ljust(6), volume.shape,end=": ")
    #x,y,z = origin
    #origin = np.array([y,x,z])
    #source_point = source_scale @ source_point
    results = r_auto @ source_point + t_auto.T
    print(results)


10N_L  (130, 88, 53): [[44642.91439093 20187.882154    -437.08533747]]
10N_R  (130, 88, 53): [[44641.69385375 19885.07730876   367.28222981]]
12N    (124, 78, 86): [[ 4.44813803e+04  2.01182872e+04 -2.79839932e+01]]
3N_L   (63, 41, 52): [[38441.02703199 19110.00714385  -604.88052262]]
3N_R   (63, 41, 52): [[38440.4640567  18970.33778696  -233.86432989]]
4N_L   (19, 23, 21): [[38590.5001316  19061.38180592  -798.56330964]]
4N_R   (19, 23, 21): [[38589.47038566 18805.91063879  -119.93242101]]
5N_L   (73, 86, 59): [[40336.44111687 21107.91557984 -1800.58912975]]
5N_R   (73, 86, 59): [[40330.69303071 19681.86452605  1987.55770799]]
6N_L   (26, 25, 26): [[40776.50822689 20152.47785547  -572.78907014]]
6N_R   (26, 25, 26): [[40774.97799986 19772.84160695   435.67261888]]
7N_L   (101, 76, 94): [[42114.67496316 22106.87496235 -1089.38049471]]
7N_R   (101, 76, 94): [[42109.81055854 20900.05774656  2116.39590969]]
7n_L   (107, 108, 150): [[41358.89765512 21436.06763441 -1144.9654538 ]]
7n_R   (1

In [ ]:
atlasV7_volume.shape

In [ ]:
%%time
ng = NumpyToNeuroglancer(atlasV7_volume, [10000, 10000, 20000])
ng.preview()
ng.init_precomputed(OUTPUT_DIR)
ng.add_segment_properties(get_segment_properties())
ng.add_downsampled_volumes()
ng.add_segmentation_mesh()